## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [278]:
import pandas as pd

In [279]:
df=pd.read_csv('Dataset/train.csv')

In [280]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [281]:
df.shape

(20800, 5)

In [282]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [283]:
###Drop Nan Values
df=df.dropna()

In [284]:
df.shape

(18285, 5)

In [285]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [286]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [287]:
## Get the Independent Features
X=df.drop('label',axis=1)

In [288]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [289]:
## Get the Dependent features
y=df['label']

In [290]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [291]:
X.shape

(18285, 4)

In [292]:
import tensorflow as tf

In [293]:
tf.__version__

'2.17.0'

In [294]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [295]:
### Vocabulary size
voc_size=10000

### Onehot Representation

In [296]:
messages=X.copy()


In [299]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [298]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [300]:
import nltk
import re
from nltk.corpus import stopwords

In [96]:
# messages = messages.reset_index(drop=True)

In [301]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [302]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [304]:
messages.reset_index(inplace=True)

In [305]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [306]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [307]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [308]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2963, 6071, 525, 4344, 3917, 5472, 7453, 6248, 4465, 6345],
 [5511, 283, 2403, 1050, 8570, 9636, 3842],
 [9259, 8161, 3190, 2478],
 [8494, 205, 4893, 5879, 4447, 433],
 [1407, 8570, 3051, 5830, 5534, 1472, 8570, 809, 1532, 1268],
 [6962,
  6125,
  4338,
  7606,
  8249,
  8176,
  5297,
  1751,
  9056,
  4157,
  6287,
  5197,
  9064,
  5232,
  3842],
 [1214, 9351, 5451, 598, 6276, 8635, 4709, 4128, 8455, 9062, 7035],
 [7234, 6445, 7844, 3460, 8049, 6459, 8176, 8641, 8455, 9062, 7035],
 [8709, 6092, 8838, 2229, 3864, 2921, 4887, 8780, 8176, 4224],
 [6848, 3707, 2954, 5738, 1437, 961, 9481, 4143],
 [7980, 4713, 9689, 4425, 87, 4246, 1532, 3445, 5781, 5991, 2310],
 [5879, 8998, 3917, 2921, 8176, 8049],
 [2839, 7792, 4852, 864, 2730, 5813, 5657, 5304, 2297],
 [1875, 5855, 5059, 3735, 4096, 9202, 7267, 8455, 9062, 7035],
 [3089, 8315, 6191, 7124, 2042, 8455, 9062, 7035],
 [3796, 984, 9452, 986, 4249, 7048, 3927, 405, 6013, 8680],
 [2767, 2864, 283],
 [131, 2670, 561, 8879, 8176, 6161, 4255,

In [257]:
corpus[2]

'truth might get fire'

In [129]:
onehot_repr[2]

[1181, 2389, 2974, 3821]

### Embedding Representation

In [309]:
sent_length=20 
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)


In [310]:
embedded_docs

array([[   0,    0,    0, ..., 6248, 4465, 6345],
       [   0,    0,    0, ..., 8570, 9636, 3842],
       [   0,    0,    0, ..., 8161, 3190, 2478],
       ...,
       [   0,    0,    0, ..., 8455, 9062, 7035],
       [   0,    0,    0, ..., 4628,  667,  866],
       [   0,    0,    0, ..., 3773, 8890, 6533]])

In [311]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2963,
       6071,  525, 4344, 3917, 5472, 7453, 6248, 4465, 6345])

In [312]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Embedding(input_dim=voc_size, output_dim=embedding_vector_features,input_length=sent_length))
# Add LSTM layer
model.add(LSTM(100))
# Add Dense layer for classification
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# Summary of the model  
model.build(input_shape=(None, sent_length))
print(model.summary())

c:\Users\Administrator\miniconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_24 (Embedding)        │ (None, 20, 40)         │       400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_24 (LSTM)                  │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 456,501 (1.74 MB)

 Trainable params: 456,501 (1.74 MB)

 Non-trainable params: 0 (0.00 B)

None


In [313]:
len(embedded_docs),y.shape

(18285, (18285,))

In [314]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [315]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [316]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [317]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64)

Epoch 1/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.8217 - loss: 0.4144 - val_accuracy: 0.9185 - val_loss: 0.1902
Epoch 2/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.9509 - loss: 0.1227 - val_accuracy: 0.9210 - val_loss: 0.1908
Epoch 3/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.9723 - loss: 0.0767 - val_accuracy: 0.9190 - val_loss: 0.2501
Epoch 4/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.9856 - loss: 0.0468 - val_accuracy: 0.9171 - val_loss: 0.3197
Epoch 5/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 0.9897 - loss: 0.0299 - val_accuracy: 0.9153 - val_loss: 0.2998
Epoch 6/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9947 - loss: 0.0184 - val_accuracy: 0.9132 - val_loss: 0.4048
Epoch 7/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.9943 - loss: 0.0211 - val_accuracy: 0.9104 - val_loss: 0.5050
Epoch 8/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.9954 - loss: 0.0155

In [318]:
# Save the model in the Keras format
model.save('fake_news_classifier.keras')

### Adding Dropout

In [182]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

c:\Users\Administrator\miniconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


### Performance Metrics And Accuracy

In [319]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step


In [321]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [322]:
from sklearn.metrics import confusion_matrix

In [323]:
confusion_matrix(y_test,y_pred)

array([[3126,  293],
       [ 271, 2345]], dtype=int64)

In [324]:
print(np.bincount(y_train))  # Check class distribution


[6942 5308]


In [325]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9065451532725767

In [326]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3419
           1       0.89      0.90      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.90      6035
weighted avg       0.91      0.91      0.91      6035



   # -------------- -------------- Test --------------- -----------

In [218]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming you have the same tokenizer as during training
tokenizer = Tokenizer(num_words=voc_size)
tokenizer.fit_on_texts(corpus)  # Fit on training corpus or load saved tokenizer

def text_to_sequence(text, tokenizer, max_len):
    sequences = tokenizer.texts_to_sequences([text])
    padded_sequences = pad_sequences(sequences, maxlen=max_len)
    return padded_sequences


In [219]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('fake_news_classifier.keras')

# Recompile the model if you need to use metrics or evaluate
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [223]:
def preprocess_text(text):
    # Implement your preprocessing steps here
    import re
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    
    # Initialize stemmer and stopwords
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    
    # Remove non-alphabetical characters
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = text.split()
    
    # Apply stemming and remove stopwords
    words = [ps.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

def text_to_sequence(text, tokenizer, max_length):
    # Convert text to sequence
    sequence = tokenizer.texts_to_sequences([text])
    # Pad sequence to ensure uniform input size
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    return pad_sequences(sequence, maxlen=max_length)

# Example news to test
new_text = "The Moon is Actually Made of Cheese—NASA Cover-Up Revealed!"

# Preprocess and predict
processed_text = preprocess_text(new_text)
sequence = text_to_sequence(processed_text, tokenizer, sent_length)
prediction = model.predict(sequence)
predicted_class = (prediction > 0.5).astype(int)

print(f"Prediction: {'Real' if predicted_class[0][0] == 1 else 'Fake'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 678ms/step
Prediction: Fake


In [224]:
raw_prediction = model.predict(sequence)
print(f"Raw prediction: {raw_prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Raw prediction: [[0.99998933]]


In [225]:
print(f"Tokenized sequence: {sequence}")

Tokenized sequence: [[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
  1252 1186  418 1216  499  127]]


In [226]:
# Evaluate model on test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model accuracy: {accuracy}")


189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9008 - loss: 0.8006
Model accuracy: 0.9075393676757812


In [227]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_text(text):
    import re
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = text.split()
    
    words = [ps.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

def text_to_sequence(text, tokenizer, max_length):
    sequence = tokenizer.texts_to_sequences([text])
    return pad_sequences(sequence, maxlen=max_length)

# Example news to test
new_text = "The Moon is Actually Made of Cheese—NASA Cover-Up Revealed!"

# Preprocess and predict
processed_text = preprocess_text(new_text)
sequence = text_to_sequence(processed_text, tokenizer, sent_length)
raw_prediction = model.predict(sequence)
predicted_class = (raw_prediction > 0.5).astype(int)

print(f"Raw prediction: {raw_prediction}")
print(f"Prediction: {'Real' if predicted_class[0][0] == 1 else 'Fake'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Raw prediction: [[0.99998933]]
Prediction: Real
